In [1]:
from torch_geometric.data import Data
from DataBuilder import QADataBuilder
from constants import (
    TRIPLES_PATH,
    ENTITIES_LABELS_PATH,
    PROPERTIES_LABELS_PATH,
    GRAPH_EMBEDDINGS_PATH,
    QUESTIONS_ANSWERS_PATH,
    QUESTIONS_EMBEDDINGS_PATH,
    NUM_EPOCHS,
)
import torch
import torch.nn.functional as F
from GNN import GCN, MLP, evaluate_qa_model
from loguru import logger
import sys

logger.remove()
logger.add(sys.stderr, level="DEBUG")

## CREATE DATA
logger.info("Creating Data object")

qa_data_builder = QADataBuilder(
    triples_path=TRIPLES_PATH,
    entities_labels_path=ENTITIES_LABELS_PATH,
    properties_labels_path=PROPERTIES_LABELS_PATH,
    embeddings_path=GRAPH_EMBEDDINGS_PATH,
    questions_answers_path=QUESTIONS_ANSWERS_PATH,
    questions_embeddings_path=QUESTIONS_EMBEDDINGS_PATH,
)

x = qa_data_builder.get_x()
edge_index = qa_data_builder.get_edge_index()
train_mask, test_mask, val_mask = qa_data_builder.get_questions_masks()
NUM_EPOCHS_PER_QUESTION = int(NUM_EPOCHS / sum(train_mask))

## TRAIN MLP
logger.info("Training MLP")
model = MLP(
    num_node_features=(2 * x.shape[1]), dim_hidden_layer=16, num_classes=2
)  # we multiply x.shape by two so as to account for question embedding
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()

for q_index, question_embedding in enumerate(
    qa_data_builder.questions_embeddings_masked(train_mask)
):  # call the questions_iterator from the instance
    question, q_embedding = question_embedding
    q_x = qa_data_builder.get_x(
        to_concat=q_embedding
    )  # add question embedding to node features embedding
    q_y = qa_data_builder.get_y(question=question)
    data = Data(x=q_x, edge_index=edge_index, y=q_y)
    if not data.validate():
        logger.error(f"Data object is not valid for question {question}")

    for epoch in range(NUM_EPOCHS_PER_QUESTION):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y, weight = torch.tensor([0.000001, 1-0.000001]))
        loss.backward()
        optimizer.step()
    logger.debug(
        f"Total Question: {(q_index + 1)}, Total Epochs: {NUM_EPOCHS_PER_QUESTION * (q_index + 1):3d}, Loss: {loss:.4f}"
    )

/Users/I302238/.local/share/virtualenvs/psmg-imkg-gnn-qa-b42qFiTl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-04 12:54:43.653 | INFO     | __main__:<module>:22 - Creating Data object
2023-08-04 12:54:53.638 | INFO     | __main__:<module>:39 - Training MLP
2023-08-04 12:54:55.795 | DEBUG    | __main__:<module>:64 - Total Question: 1, Total Epochs:  96, Loss: 0.1575
2023-08-04 12:54:57.920 | DEBUG    | __main__:<module>:64 - Total Question: 2, Total Epochs: 192, Loss: 0.1105
2023-08-04 12:55:00.053 | DEBUG    | __main__:<module>:64 - Total Question: 3, Total Epochs: 288, Loss: 0.1115
2023-08-04 12:55:02.194 | DEBUG    | __main__:<module>:64 - Total Question: 4, Total Epochs: 384, Loss: 0.2434
2023-08-04 12:55:04.330 | DEBUG    | __main__:<module>:64 - Total Question: 5, Total Epochs: 480, Loss

KeyboardInterrupt: 

In [8]:
evaluate_qa_model(model=model, qa_data_builder=qa_data_builder, mask=train_mask)

2023-08-04 12:53:10.477 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:10.590 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:10.710 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:10.821 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:10.939 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.049 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.158 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.266 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.372 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.478 | DEBUG    | __main__:evaluate_qa_model:29 - Could not predict any answer
2023-08-04 12:53:11.

KeyboardInterrupt: 

In [7]:
def _predict_answer(model, data):
    """
    Returns the predicted answer and node index.
    """
    return model(data).max(dim=1)[0].argmax().item()

def evaluate_qa_model(model, qa_data_builder, mask):
    model.eval()
    correct_predictions = 0
    for q_index, question_embedding in enumerate(
        qa_data_builder.questions_embeddings_masked(mask)
    ):
        question, q_embedding = question_embedding
        x_q = qa_data_builder.get_x(
            to_concat=q_embedding
        )  # adding the question embedding to the node embeddings
        y_q = qa_data_builder.get_y(question=question)
        data = Data(x=x_q, edge_index=qa_data_builder.get_edge_index(), y=y_q)
        pred_node_idx = _predict_answer(model, data)
        actual_node_idx = qa_data_builder.get_node_index_for_question_answer(question)
        if pred_node_idx == actual_node_idx:
            logger.debug(f"Correctly predicted answer to question {question}.")
            correct_predictions += 1
        elif pred_node_idx != torch.tensor(0):
            logger.debug(
                f"Question: {question}. Predicted answer = {qa_data_builder.index_to_entity[pred_node_idx]}, Actual answer: {qa_data_builder.index_to_entity[actual_node_idx]}"
            )
        else:
            logger.debug(f"Could not predict any answer")
    return correct_predictions / (q_index + 1)